In [1]:
from decouple import config
from qiskit import IBMQ
from datetime import datetime
import pprint
import json

In [2]:
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(hub='strangeworks-hub', group='qc-com', project='runtime')
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='strangeworks-hub', group='qc-com', project='runtime')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='strangeworks-hub', group='qc-com', project='runtime')>]


In [3]:
import pandas as pd

df = pd.read_csv('qiskit_runtime/qka/aux_file/dataset_graph7.csv',sep=',', header=None) # alterative problem: dataset_graph10.csv
data = df.values

In [4]:
import numpy as np

# choose number of training and test samples per class:
num_train = 10
num_test = 10

# extract training and test sets and sort them by class label
train = data[:2*num_train, :]
test = data[2*num_train:2*(num_train+num_test), :]

ind=np.argsort(train[:,-1])
x_train = train[ind][:,:-1]
y_train = train[ind][:,-1]

ind=np.argsort(test[:,-1])
x_test = test[ind][:,:-1]
y_test = test[ind][:,-1]

In [5]:
from qiskit_runtime.qka import FeatureMap

d = np.shape(data)[1]-1                                         # feature dimension is twice the qubit number

em = [[0,2],[3,4],[2,5],[1,4],[2,3],[4,6]]                      # we'll match this to the 7-qubit graph
# em = [[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]]  # we'll match this to the 10-qubit graph

fm = FeatureMap(feature_dimension=d, entangler_map=em)          # define the feature map
initial_point = [0.1]                                           # set the initial parameter for the feature map

In [6]:
from qiskit.tools.visualization import circuit_drawer
circuit_drawer(fm.construct_circuit(x=x_train[0], parameters=initial_point),
               output='text', fold=200)

┌──────────┐   ┌──────────────┐ ┌────────────┐                                                 
q_0: ┤ RY(-0.1) ├─■─┤ RZ(-0.38383) ├─┤ RX(3.3378) ├─────────────────────────────────────────────────
     ├──────────┤ │ └──────────────┘ └────────────┘┌──────────────┐ ┌──────────────┐                
q_1: ┤ RY(-0.1) ├─┼────────────────────────■───────┤ RZ(-0.11811) ├─┤ RX(-0.20449) ├────────────────
     ├──────────┤ │                        │       └──────────────┘ └┬────────────┬┘┌─────────────┐ 
q_2: ┤ RY(-0.1) ├─■────────■───────────────┼──────────────■──────────┤ RZ(3.4802) ├─┤ RX(0.15495) ├─
     ├──────────┤          │               │              │         ┌┴────────────┤ ├─────────────┴┐
q_3: ┤ RY(-0.1) ├─■────────┼───────────────┼──────────────■─────────┤ RZ(0.34764) ├─┤ RX(-0.54085) ├
     ├──────────┤ │        │               │                        ├─────────────┤ ├──────────────┤
q_4: ┤ RY(-0.1) ├─■────────┼───────────────■──────────────■─────────┤ RZ(0.34312) ├─┤ RX(-0.14015) ├
     ├──────────┤          │        ┌─────────────┐       │        ┌┴─────────────┴┐└──────────────┘
q_5: ┤ RY(-0.1) ├──────────■────────┤ RZ(0.51497) ├───────┼────────┤ RX(-0.029293) ├────────────────
     ├──────────┤                   └─────────────┘       │        └┬──────────────┤┌─────────────┐ 
q_6: ┤ RY(-0.1) ├─────────────────────────────────────────■─────────┤ RZ(-0.42725) ├┤ RX(0.44115) ├─
     └──────────┘                                                   └──────────────┘└─────────────┘

In [7]:
C = 1                                                           # SVM soft-margin penalty
maxiters = 10                                                   # number of SPSA iterations



In [8]:
initial_layout = [10, 11, 12, 13, 14, 15, 16]                   # see figure above for the 7-qubit graph
# initial_layout = [9, 8, 11, 14, 16, 19, 22, 25, 24, 23]       # see figure above for the 10-qubit graph

In [9]:
print(provider.runtime.program('quantum-kernel-alignment'))



quantum-kernel-alignment:
  Name: quantum-kernel-alignment
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Version: 1
  Creation date: 2021-05-06T14:56:53.000000
  Max execution time: 28800
  Input parameters:
    - feature_map:
      Description: An instance of FeatureMap in dictionary format used to map classical data into a quantum state space.
      Type: dict
      Required: True
    - data:
      Description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      Type: numpy.ndarray
      Required: True
    - labels:
      Description: Nx1 array of +/-1 labels of the N training samples.
      Type: numpy.ndarray
      Required: True
    - initial_kernel_parameters:
      Description: Initial parameters of the quantum kernel. If not specified, an array of randomly generated numbers is used.
      Type: numpy.ndarray
      Required: False
    -

In [10]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}\n")

In [11]:
# backend = provider.get_backend('ibmq_qasm_simulator')
backend = provider.get_backend('ibm_nairobi')


In [12]:
program_inputs = {
    'feature_map': fm,
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_point,
    'maxiters': maxiters,
    'C': C,
    'initial_layout': initial_layout
}

options = {'backend_name': backend.name()}

job = provider.runtime.run(program_id="quantum-kernel-alignment",
                              options=options,
                              inputs=program_inputs,
                              callback=interim_result_callback,
                              )

print(job.job_id())
result = job.result()

c3kb47cn850hs8tm1940


RuntimeJobFailureError: 'Unable to retrieve result for job c3kb47cn850hs8tm1940. Job has failed:\n2021-07-09T16:31:37.515199768-04:00 07/09/2021 08:31:37 PM - asyncio - DEBUG Using selector: EpollSelector\n2021-07-09T16:31:37.525462817-04:00 nats_client.connect:DEBUG:2021-07-09 20:31:37,525: Connecting to server at tls://nats-cluster.nats-io.svc.cluster.local:4222\n2021-07-09T16:31:37.536995325-04:00 runtime_provider.__init__:DEBUG:2021-07-09 20:31:37,536: QuantumProgramProvider initialization completed. Program ID=quantum-kernel-alignment, execution ID=c3kb47cn850hs8tm1940\n2021-07-09T16:31:37.537246063-04:00 nats_client.connect:DEBUG:2021-07-09 20:31:37,537: Connecting to server at tls://nats.quantum-computing.ibm.com:4222\n2021-07-09T16:31:38.848297028-04:00 07/09/2021 08:31:38 PM - qiskit.transpiler.runningpassmanager - INFO Pass: SetLayout - 0.02313 (ms)\n2021-07-09T16:31:38.848436585-04:00 07/09/2021 08:31:38 PM - qiskit.transpiler.runningpassmanager - INFO Pass: FullAncillaAlloc